In [1]:
!pip install rouge

In [2]:
import json
import torch
from datasets import Dataset
from tqdm import tqdm
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator
)

2025-09-20 12:31:46.746738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758371507.133015      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758371507.228730      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def load_vatika_data(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    contexts, questions, answers = [], [], []
    for domain in data["domains"]:
        for item in domain["contexts"]:
            context = item["context"]
            for qa in item["qas"]:
                contexts.append(context)
                questions.append(qa["question"])
                answers.append(qa["answer"])
    return {"context": contexts, "question": questions, "answers": answers}

train_path = "/kaggle/input/vatika/train.json"
train_data = load_vatika_data(train_path)

In [4]:
def prepare_features(examples, tokenizer):
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=512,
        stride=128,
        return_offsets_mapping=True,
        padding="max_length"
    )
    start_positions, end_positions = [], []
    for i, offsets in enumerate(tokenized["offset_mapping"]):
        context = examples["context"][i]
        answer = examples["answers"][i]
        answer_start = context.find(answer)
        answer_end = answer_start + len(answer)

        start = end = 0
        for idx, (s, e) in enumerate(offsets):
            if s <= answer_start < e:
                start = idx
            if s < answer_end <= e:
                end = idx
                break
        start_positions.append(start)
        end_positions.append(end)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized

tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
model = AutoModelForQuestionAnswering.from_pretrained("google/muril-base-cased")

dataset = Dataset.from_dict(train_data)
tokenized_dataset = dataset.map(lambda x: prepare_features(x, tokenizer), batched=True)


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/13092 [00:00<?, ? examples/s]

In [5]:
training_args = TrainingArguments(
    output_dir="./muril-vatika",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="no",
    logging_strategy="no",
    report_to="none",
    disable_tqdm=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

trainer.train()

/tmp/ipykernel_36/891834965.py:13: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'train_runtime': 1743.8363, 'train_samples_per_second': 15.015, 'train_steps_per_second': 0.939, 'train_loss': 4.831488119085775, 'epoch': 2.0}


TrainOutput(global_step=1638, training_loss=4.831488119085775, metrics={'train_runtime': 1743.8363, 'train_samples_per_second': 15.015, 'train_steps_per_second': 0.939, 'train_loss': 4.831488119085775, 'epoch': 2.0})

In [6]:
def load_test(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)

def predict_answers(model, tokenizer, test_data):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for domain in test_data["domains"]:
        for ctx in domain["contexts"]:
            context = ctx["context"]
            for qa in ctx["qas"]:
                question = qa["question"]
                inputs = tokenizer(
                    question,
                    context,
                    return_tensors="pt",
                    truncation=True,
                    max_length=512,
                    padding="max_length"
                ).to(device)

                with torch.no_grad():
                    outputs = model(**inputs)
                    start = torch.argmax(outputs.start_logits)
                    end = torch.argmax(outputs.end_logits)
                    tokens = inputs["input_ids"][0][start:end+1]
                    answer = tokenizer.decode(tokens, skip_special_tokens=True)
                    qa["answer"] = answer.strip()
    return test_data

test_path = "/kaggle/input/vatika/test-B.json"
test_data = load_test(test_path)
predictions = predict_answers(model, tokenizer, test_data)

with open("/kaggle/working/Test-B-predicted.json", "w", encoding="utf-8") as f:
    json.dump(predictions, f, ensure_ascii=False, indent=2)

print("Saved predictions to Test-B-predicted.json")

Saved predictions to Test-B-predicted.json


In [7]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Load validation and test-A gold data
val_data = load_test("/kaggle/input/vatika/validation.json")
testA_data = load_test("/kaggle/input/vatika/test-A-gold.json")

def extract_references(data):
    refs, qs, ctxs = [], [], []
    for domain in data["domains"]:
        for ctx in domain["contexts"]:
            for qa in ctx["qas"]:
                refs.append(qa["answer"])
                qs.append(qa["question"])
                ctxs.append(ctx["context"])
    return refs, qs, ctxs

def get_predictions(qs, ctxs):
    preds = []
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    with torch.no_grad():
        for q, c in zip(qs, ctxs):
            inputs = tokenizer(q, c, return_tensors="pt", truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            start = torch.argmax(outputs.start_logits)
            end = torch.argmax(outputs.end_logits) + 1
            tokens = inputs["input_ids"][0][start:end]
            ans = tokenizer.decode(tokens, skip_special_tokens=True).strip()
            preds.append(ans)
    return preds

In [8]:
def compute_metrics(references, predictions, debug=False):
    # F1
    f1s = []
    for ref, pred in zip(references, predictions):
        ref_tokens, pred_tokens = ref.split(), pred.split()
        common = set(ref_tokens) & set(pred_tokens)
        if not common:
            f1s.append(0.0)
        else:
            precision = len(common) / len(pred_tokens) if pred_tokens else 0
            recall = len(common) / len(ref_tokens) if ref_tokens else 0
            f1s.append(2 * precision * recall / (precision + recall) if precision + recall > 0 else 0)
    avg_f1 = sum(f1s) / len(f1s)

    # Exact Match
    em = sum([1 if r.strip() == p.strip() else 0 for r, p in zip(references, predictions)]) / len(references)

    # BLEU (safe)
    bleu = 0
    for r, p in zip(references, predictions):
        if p.strip():  # skip empty predictions
            bleu += sentence_bleu([r.split()], p.split())
    bleu /= len(references)

    # ROUGE-L (safe)
    rouge = Rouge()
    rouge_scores = []
    empty_cases = []
    for i, (r, p) in enumerate(zip(references, predictions)):
        if not p.strip():  # empty prediction
            rouge_scores.append(0.0)
            empty_cases.append((i, r, p))
        else:
            rouge_scores.append(rouge.get_scores(p, r)[0]['rouge-l']['f'])
    rouge_l = sum(rouge_scores) / len(references)

    if debug and empty_cases:
        print("\nEmpty predictions found in these cases:")
        for i, r, p in empty_cases[:10]:  # show first 10 only
            print(f"Index {i} | Ref: {r} | Pred: '{p}'")

    return avg_f1, em, bleu, rouge_l


In [9]:
val_refs, val_qs, val_ctxs = extract_references(val_data)
val_preds = get_predictions(val_qs, val_ctxs)
val_f1, val_em, val_bleu, val_rouge = compute_metrics(val_refs, val_preds, debug=True)

print("\n Validation Metrics")
print(f"F1 Score: {val_f1:.4f}")
print(f"Exact Match: {val_em:.4f}")
print(f"BLEU Score: {val_bleu:.4f}")
print(f"ROUGE-L Score: {val_rouge:.4f}")

# Test-A Metrics
testA_refs, testA_qs, testA_ctxs = extract_references(testA_data)
testA_preds = get_predictions(testA_qs, testA_ctxs)
testA_f1, testA_em, testA_bleu, testA_rouge = compute_metrics(testA_refs, testA_preds, debug=True)

print("\n Test-A Metrics")
print(f"F1 Score: {testA_f1:.4f}")
print(f"Exact Match: {testA_em:.4f}")
print(f"BLEU Score: {testA_bleu:.4f}")
print(f"ROUGE-L Score: {testA_rouge:.4f}")


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_


Empty predictions found in these cases:
Index 8 | Ref: हाँ, कुरुक्षेत्र कुंड में सुरक्षा व्यवस्थाएँ प्रमुख धार्मिक अवसरों पर सुरक्षा के उचित प्रबंध किए जाते हैं। | Pred: ''
Index 21 | Ref: नहीं, दुर्गा कुंड के पास पार्किंग की सुविधा उपलब्ध नहीं है। | Pred: ''
Index 26 | Ref: हाँ, पांडव कुंड में स्नान करने से आत्मा की शुद्धि से जीवन के पाप समाप्त होते हैं। | Pred: ''
Index 31 | Ref: हाँ, दुर्गा कुंड में नवरात्रि के दौरान विशेष पूजा और दशहरा के मेले का आयोजन होता है। | Pred: ''
Index 34 | Ref: दुर्गा कुंड में नवरात्रि का आयोजन 9 दिनों तक चलता है। | Pred: ''
Index 35 | Ref: हाँ, क्रीं कुंड तक पहुँचने के लिए बस, ऑटो-रिक्शा और टैक्सी की सुविधा उपलब्ध है। | Pred: ''
Index 36 | Ref: हाँ, क्रीं कुंड से जुड़े सार्वजनिक परिवहन मौजूद हैं। इसके अलावा, स्थानीय बस सेवाओं का भी उपयोग किया जा सकता है। | Pred: ''
Index 37 | Ref: नहीं, क्रीं कुंड तक पहुँचने के लिए सार्वजनिक परिवहन सबसे अच्छा विकल्प है। | Pred: ''
Index 38 | Ref: हाँ, नाग कुआँ में स्नान करने की अनुमति है। | Pred: ''
Index 47 | Ref: नहीं